# [VGG]

*KU LeeDongGyu*

### Contents

1. Almost Original VGG19 & Data Import
2. For Size = 64,
```
1) My VGG11 (Pretraining)
2) My VGG16 (Pretrained)
3) My VGG19 (Pretrained)
4) My VGG11 (Convergence Speed Comparison)
```
3. For Size = 48,
```
1) My VGG11 (Pretraining)
2) My VGG16 (Pretrained)
3) My VGG19 (Pretrained)
4) My VGG11 (Convergence Speed Comparison)
```
4. For Size = 48, No Early Stopping
```
1) Epoch = 50
      (1) My VGG11
      (2) My VGG16
      (3) My VGG19
2) Epoch = 100
      (1) My VGG11
      (2) My VGG16
      (3) My VGG19
```
5. Model Save & Import

### Install Packages

### Module

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [5]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [7]:
from lrn import LRN
from f1score import macro_f1score,weighted_f1score

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file, to_categorical

In [9]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

## 1. Almost Original VGG19 & Data Import
---

In [0]:
# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48,3]) 
x_valid = np.array(x_valid).reshape([-1,48,48,3]) 
x_test = np.array(x_test).reshape([-1,48,48,3]) 

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size = 64

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,3],dtype="float32")
for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_train = x_train / 255

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,3])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,3],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
 x_valid = x_valid / 255

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,3])
x_test_zoom = np.zeros([x_test.shape[0],size,size,3],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) / 255

In [0]:
x_test = x_test / 255

In [19]:
# VGG19 모형과 관련된 초기값이 저장되어 있는 링크. 접속시 바로 다운됨. FC층을 어떻게 처리할 건지에 따라 두가지 초기값들이 있음.
# 이 값은 ImageNet 데이터를 가지고 학습한 초기값임.
# 내가 쓸 모형은 이 초기값이 필요 없기 때문에, 내가 초기값을 뽑아내는 작업을 해야한다.

# 다시한번 얘기하지만, 아래의 두 변수는 내가 아래의 코드에서 쓸 변수는 아님. 나중의 혹시모를 상황을 대비해 남겨놓음.
WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

# -*- coding: utf-8 -*-
'''VGG19 model for Keras.
# Reference:
- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
'''
('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
"""Instantiates the VGG19 architecture.
 Optionally loads weights pre-trained on ImageNet. Note that when using TensorFlow,
 for best performance you should set
 `image_data_format="channels_last"` in your Keras config
 at ~/.keras/keras.json.
 The model and the weights are compatible with both
 TensorFlow and Theano. The data format
 convention used by the model is the one
 specified in your Keras config file.

 # Arguments
     include_top: whether to include the 3 fully-connected
         layers at the top of the network.
     weights: one of `None` (random initialization)
         or "imagenet" (pre-training on ImageNet).
     input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
         to use as image input for the model.
     input_shape: optional shape tuple, only to be specified
         if `include_top` is False (otherwise the input shape
         has to be `(224, 224, 3)` (with `channels_last` data format)
         or `(3, 224, 244)` (with `channels_first` data format).
         It should have exactly 3 inputs channels,
         and width and height should be no smaller than 48.
         E.g. `(200, 200, 3)` would be one valid value.
     pooling: Optional pooling mode for feature extraction
         when `include_top` is `False`.
         - `None` means that the output of the model will be
             the 4D tensor output of the
             last convolutional layer.
         - `avg` means that global average pooling
             will be applied to the output of the
             last convolutional layer, and thus
             the output of the model will be a 2D tensor.
         - `max` means that global max pooling will
             be applied.
     classes: optional number of classes to classify images
         into, only to be specified if `include_top` is True, and
         if no `weights` argument is specified.
 # Returns
     A Keras model instance.
 """


'Instantiates the VGG19 architecture.\n Optionally loads weights pre-trained on ImageNet. Note that when using TensorFlow,\n for best performance you should set\n `image_data_format="channels_last"` in your Keras config\n at ~/.keras/keras.json.\n The model and the weights are compatible with both\n TensorFlow and Theano. The data format\n convention used by the model is the one\n specified in your Keras config file.\n\n # Arguments\n     include_top: whether to include the 3 fully-connected\n         layers at the top of the network.\n     weights: one of `None` (random initialization)\n         or "imagenet" (pre-training on ImageNet).\n     input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)\n         to use as image input for the model.\n     input_shape: optional shape tuple, only to be specified\n         if `include_top` is False (otherwise the input shape\n         has to be `(224, 224, 3)` (with `channels_last` data format)\n         or `(3, 224, 244)` (with 

In [0]:
# VGG19를 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.

def VGG19(input_shape=(224, 224, 3), classes=7, include_top=True, pooling=None, weights=None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    if weights == 'imagenet': # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH, cache_subdir='models', file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP, cache_subdir='models',
                                    file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path)  # 경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model

In [21]:
model = VGG19(input_shape=(224, 224, 3), classes=7)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [22]:
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## 2. For Size = 64,
---

### 1) My VGG11 (Pretraining)

In [0]:
# 논문에서 VGG19 의 FC층은 VGG11로 pretraining하여 얻은 초기치로 설정하였음.
# 기존의 VGG11을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# 다음의 절차로 모형을 개조한다.

# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.

def VGG11(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu',name='fc1')(x)
        x = Dense(256, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
#내 데이터 맞춤형 모형
model = VGG11(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)

In [46]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 16, 16, 32) 

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [48]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 3s 115us/sample - loss: 1.8177 - acc: 0.2498 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8029 - val_acc: 0.2786 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 3s 89us/sample - loss: 1.6223 - acc: 0.3549 - macro_f1score: 0.0974 - weighted_f1score: 0.0203 - val_loss: 1.4847 - val_acc: 0.4260 - val_macro_f1score: 0.1677 - val_weighted_f1score: 0.0354
Epoch 3/100
28698/28698 [==============================] - 3s 89us/sample - loss: 1.4430 - acc: 0.4362 - macro_f1score: 0.1872 - weighted_f1score: 0.0351 - val_loss: 1.3827 - val_acc: 0.4756 - val_macro_f1score: 0.1775 - val_weighted_f1score: 0.0332
Epoch 4/100
28698/28698 [==============================] - 3s 88us/sample - loss: 1.3369 - acc: 0.4845 - macro_f1score: 0.2264 - weighted_f1score: 0.0421 - val_loss: 1.3293 - val_acc: 

In [49]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 0s 60us/sample - loss: 2.9833 - acc: 0.5270 - macro_f1score: 0.4738 - weighted_f1score: 0.0738

Accuracy: 0.5270, Macro F1 Score: 0.4738, Weighted F1 Score: 0.0738


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

In [51]:
W

[array([[[[-5.53251095e-02,  8.42718780e-02,  2.55060732e-01,
            6.52565956e-02,  4.36374843e-02,  1.49025887e-01,
           -2.98429817e-01,  1.37789965e-01],
          [-5.62942661e-02, -1.04226284e-01,  2.57335097e-01,
           -1.74122319e-01,  3.25474948e-01,  1.37393907e-01,
           -1.90222904e-01,  1.32181287e-01],
          [-8.91920328e-02, -2.10791722e-01, -1.81012020e-01,
           -1.90785572e-01,  3.52549702e-02,  2.34128356e-01,
           -3.24729569e-02,  3.97629244e-03]],
 
         [[-5.21460222e-03,  1.18406199e-01, -2.92174548e-01,
           -2.76503801e-01,  6.05570078e-02, -2.37071097e-01,
           -7.29150698e-02,  2.55067021e-01],
          [ 2.43245855e-01,  2.59282172e-01,  8.91236365e-02,
           -2.85209477e-01,  4.62313443e-02,  5.24091013e-02,
           -7.03401566e-02, -3.97660434e-02],
          [ 6.37549236e-02, -8.14780667e-02, -1.47266731e-01,
           -6.04677163e-02,  3.06224823e-01,  1.62422106e-01,
           -2.61379480e

In [52]:
model.weights # 참고. get_weights()와 weights는 다른 매서드임을 알자.

[<tf.Variable 'block1_conv1_4/kernel:0' shape=(3, 3, 3, 8) dtype=float32>,
 <tf.Variable 'block1_conv1_4/bias:0' shape=(8,) dtype=float32>,
 <tf.Variable 'block2_conv1_4/kernel:0' shape=(3, 3, 8, 16) dtype=float32>,
 <tf.Variable 'block2_conv1_4/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'block3_conv1_4/kernel:0' shape=(3, 3, 16, 32) dtype=float32>,
 <tf.Variable 'block3_conv1_4/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block3_conv2_4/kernel:0' shape=(3, 3, 32, 32) dtype=float32>,
 <tf.Variable 'block3_conv2_4/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block4_conv1_4/kernel:0' shape=(3, 3, 32, 64) dtype=float32>,
 <tf.Variable 'block4_conv1_4/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block4_conv2_4/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block4_conv2_4/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block5_conv1_4/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block5_conv1_4/bias:0' shape=(64,) dtype=float32>,
 <t

In [53]:
# 형태는 이렇게 생김
np.array(W).shape

(22,)

### 2) My VGG16 (Pretrained)

In [0]:
# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.


def VGG16(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
model = VGG16(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)

In [62]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [64]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 4s 155us/sample - loss: 1.7989 - acc: 0.2520 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.7938 - val_acc: 0.2471 - val_macro_f1score: 6.5681e-04 - val_weighted_f1score: 7.1839e-05
Epoch 2/100
28698/28698 [==============================] - 4s 130us/sample - loss: 1.6455 - acc: 0.3420 - macro_f1score: 0.0814 - weighted_f1score: 0.0149 - val_loss: 1.4865 - val_acc: 0.4154 - val_macro_f1score: 0.1677 - val_weighted_f1score: 0.0313
Epoch 3/100
28698/28698 [==============================] - 4s 130us/sample - loss: 1.4355 - acc: 0.4383 - macro_f1score: 0.1822 - weighted_f1score: 0.0341 - val_loss: 1.3653 - val_acc: 0.4653 - val_macro_f1score: 0.1938 - val_weighted_f1score: 0.0361
Epoch 4/100
28698/28698 [==============================] - 4s 131us/sample - loss: 1.3316 - acc: 0.4832 - macro_f1score: 0.2311 - weighted_f1score: 0.0426 - val_loss: 1.3006 - val_ac

In [65]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 61us/sample - loss: 1.5322 - acc: 0.5702 - macro_f1score: 0.4947 - weighted_f1score: 0.0788

Accuracy: 0.5702, Macro F1 Score: 0.4947, Weighted F1 Score: 0.0788


### 3) My VGG19 (Pretrained)

In [0]:
# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.


def VGG19(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
#내 데이터 맞춤형 모형
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)


In [56]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [58]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 5s 165us/sample - loss: 1.8107 - acc: 0.2473 - macro_f1score: 2.3483e-04 - weighted_f1score: 6.0543e-05 - val_loss: 1.7857 - val_acc: 0.2510 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 4s 140us/sample - loss: 1.7350 - acc: 0.2827 - macro_f1score: 0.0282 - weighted_f1score: 0.0038 - val_loss: 1.6831 - val_acc: 0.3179 - val_macro_f1score: 0.0370 - val_weighted_f1score: 0.0042
Epoch 3/100
28698/28698 [==============================] - 4s 140us/sample - loss: 1.5652 - acc: 0.3816 - macro_f1score: 0.1320 - weighted_f1score: 0.0246 - val_loss: 1.5138 - val_acc: 0.4048 - val_macro_f1score: 0.1685 - val_weighted_f1score: 0.0331
Epoch 4/100
28698/28698 [==============================] - 4s 139us/sample - loss: 1.4459 - acc: 0.4294 - macro_f1score: 0.1720 - weighted_f1score: 0.0327 - val_loss: 1.4191 - val_ac

In [59]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 0s 69us/sample - loss: 1.5933 - acc: 0.5479 - macro_f1score: 0.4759 - weighted_f1score: 0.0768

Accuracy: 0.5479, Macro F1 Score: 0.4759, Weighted F1 Score: 0.0768


### 4) My VGG11 (Convergence Speed Comparison)

In [0]:
# Vgg11에서, pretraining 된 fc층을 가지고 VGG 11을 재학습 시킬때, 얼마나 적은 epoch만에 수렴하는지를 보기 위해 만들었다.


# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.

def VGG11_init(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
model = VGG11_init(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)

In [47]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 16, 16, 32) 

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [49]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 3s 104us/sample - loss: 1.6883 - acc: 0.3175 - macro_f1score: 0.0598 - weighted_f1score: 0.0119 - val_loss: 1.5161 - val_acc: 0.4057 - val_macro_f1score: 0.1624 - val_weighted_f1score: 0.0312
Epoch 2/100
28698/28698 [==============================] - 3s 88us/sample - loss: 1.4595 - acc: 0.4341 - macro_f1score: 0.1746 - weighted_f1score: 0.0330 - val_loss: 1.4280 - val_acc: 0.4416 - val_macro_f1score: 0.1783 - val_weighted_f1score: 0.0342
Epoch 3/100
28698/28698 [==============================] - 3s 88us/sample - loss: 1.3320 - acc: 0.4854 - macro_f1score: 0.2329 - weighted_f1score: 0.0430 - val_loss: 1.3091 - val_acc: 0.4887 - val_macro_f1score: 0.2689 - val_weighted_f1score: 0.0484
Epoch 4/100
28698/28698 [==============================] - 3s 88us/sample - loss: 1.2428 - acc: 0.5242 - macro_f1score: 0.2950 - weighted_f1score: 0.0527 - val_loss: 1.2589 - val_acc: 0.5196 - val_mac

In [50]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 50us/sample - loss: 1.8121 - acc: 0.5432 - macro_f1score: 0.4770 - weighted_f1score: 0.0772

Accuracy: 0.5432, Macro F1 Score: 0.4770, Weighted F1 Score: 0.0772


## 3. For Size =48,
---

### 1) My VGG11 (Pretraining)

In [0]:
# 오리지널 데이터에 대해 다루어본다.

def VGG11(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu',name='fc1')(x)
        x = Dense(256, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [20]:
#내 데이터 맞춤형 모형
model = VGG11(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [21]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 12, 12, 32) 

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [25]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 7s 250us/sample - loss: 1.8052 - acc: 0.2560 - macro_f1score: 0.0040 - weighted_f1score: 0.0010 - val_loss: 1.7466 - val_acc: 0.3090 - val_macro_f1score: 0.0756 - val_weighted_f1score: 0.0186
Epoch 2/100
28698/28698 [==============================] - 5s 190us/sample - loss: 1.5951 - acc: 0.3732 - macro_f1score: 0.1140 - weighted_f1score: 0.0233 - val_loss: 1.5200 - val_acc: 0.4110 - val_macro_f1score: 0.1469 - val_weighted_f1score: 0.0289
Epoch 3/100
28698/28698 [==============================] - 5s 190us/sample - loss: 1.4619 - acc: 0.4302 - macro_f1score: 0.1687 - weighted_f1score: 0.0325 - val_loss: 1.4277 - val_acc: 0.4388 - val_macro_f1score: 0.1608 - val_weighted_f1score: 0.0305
Epoch 4/100
28698/28698 [==============================] - 5s 190us/sample - loss: 1.3731 - acc: 0.4665 - macro_f1score: 0.2062 - weighted_f1score: 0.0388 - val_loss: 1.3818 - val_acc: 0.4659 - val_

In [26]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 0s 85us/sample - loss: 2.2662 - acc: 0.5187 - macro_f1score: 0.4677 - weighted_f1score: 0.0752

Accuracy: 0.5187, Macro F1 Score: 0.4677, Weighted F1 Score: 0.0752


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

In [28]:
W

[array([[[[-0.1510465 ,  0.23531386, -0.2207197 ,  0.13309611,
           -0.0603352 , -0.3178591 , -0.11177752,  0.17625943],
          [ 0.03597316, -0.11706031, -0.0535823 ,  0.26302415,
           -0.12823464,  0.11383588, -0.20325002,  0.08423396],
          [ 0.19377951,  0.09417833, -0.03279843,  0.22623624,
           -0.14550078,  0.04951933, -0.08644174, -0.2758258 ]],
 
         [[ 0.14982955, -0.01524286, -0.06072063,  0.1583087 ,
           -0.11703017, -0.20971274, -0.23559268,  0.09684745],
          [ 0.17019747,  0.11922974,  0.08854751, -0.27740335,
            0.26332873, -0.09203462, -0.06980166,  0.0771119 ],
          [-0.26574698, -0.05068942, -0.25282478, -0.2930892 ,
            0.25807473, -0.18661265, -0.11139636,  0.10976446]],
 
         [[-0.10811165,  0.24608824,  0.05893483, -0.18050395,
            0.21645854,  0.11834465,  0.01841084, -0.03133779],
          [-0.1756068 , -0.10027721, -0.01073667,  0.19899067,
            0.14594008,  0.14379402,  0.06

In [29]:
model.weights # 참고. get_weights()와 weights는 다른 매서드임을 알자.

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 8) dtype=float32>,
 <tf.Variable 'block1_conv1/bias:0' shape=(8,) dtype=float32>,
 <tf.Variable 'block2_conv1/kernel:0' shape=(3, 3, 8, 16) dtype=float32>,
 <tf.Variable 'block2_conv1/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'block3_conv1/kernel:0' shape=(3, 3, 16, 32) dtype=float32>,
 <tf.Variable 'block3_conv1/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block3_conv2/kernel:0' shape=(3, 3, 32, 32) dtype=float32>,
 <tf.Variable 'block3_conv2/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block4_conv1/kernel:0' shape=(3, 3, 32, 64) dtype=float32>,
 <tf.Variable 'block4_conv1/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block4_conv2/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block4_conv2/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block5_conv1/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block5_conv1/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block5_conv2/ker

In [30]:
# 형태는 이렇게 생김
np.array(W).shape

(22,)

### 2) My VGG16 (Pretrained)

In [0]:
def VGG16(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
model = VGG16(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [63]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [65]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 7s 247us/sample - loss: 1.7817 - acc: 0.2614 - macro_f1score: 0.0121 - weighted_f1score: 0.0015 - val_loss: 1.6914 - val_acc: 0.3023 - val_macro_f1score: 0.0395 - val_weighted_f1score: 0.0045
Epoch 2/100
28698/28698 [==============================] - 6s 223us/sample - loss: 1.5995 - acc: 0.3672 - macro_f1score: 0.1163 - weighted_f1score: 0.0210 - val_loss: 1.4773 - val_acc: 0.4269 - val_macro_f1score: 0.1687 - val_weighted_f1score: 0.0321
Epoch 3/100
28698/28698 [==============================] - 6s 224us/sample - loss: 1.4471 - acc: 0.4354 - macro_f1score: 0.1739 - weighted_f1score: 0.0329 - val_loss: 1.3846 - val_acc: 0.4636 - val_macro_f1score: 0.1993 - val_weighted_f1score: 0.0386
Epoch 4/100
28698/28698 [==============================] - 6s 222us/sample - loss: 1.3496 - acc: 0.4794 - macro_f1score: 0.2245 - weighted_f1score: 0.0413 - val_loss: 1.3333 - val_acc: 0.4726 - val_

In [66]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 90us/sample - loss: 1.5601 - acc: 0.5368 - macro_f1score: 0.4565 - weighted_f1score: 0.0731

Accuracy: 0.5368, Macro F1 Score: 0.4565, Weighted F1 Score: 0.0731


### 3) My VGG19 (Pretrained)

In [0]:
def VGG19(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='block5_pool')(x) 
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.   

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
#내 데이터 맞춤형 모형
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [33]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [35]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 7s 250us/sample - loss: 1.8140 - acc: 0.2466 - macro_f1score: 2.4456e-04 - weighted_f1score: 6.1140e-05 - val_loss: 1.8039 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 7s 232us/sample - loss: 1.7553 - acc: 0.2734 - macro_f1score: 0.0147 - weighted_f1score: 0.0024 - val_loss: 1.6777 - val_acc: 0.3204 - val_macro_f1score: 0.0765 - val_weighted_f1score: 0.0107
Epoch 3/100
28698/28698 [==============================] - 7s 233us/sample - loss: 1.6120 - acc: 0.3528 - macro_f1score: 0.1161 - weighted_f1score: 0.0217 - val_loss: 1.5489 - val_acc: 0.3948 - val_macro_f1score: 0.1463 - val_weighted_f1score: 0.0268
Epoch 4/100
28698/28698 [==============================] - 7s 232us/sample - loss: 1.4943 - acc: 0.4098 - macro_f1score: 0.1657 - weighted_f1score: 0.0311 - val_loss: 1.4405 - val_ac

In [36]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 0s 91us/sample - loss: 1.9061 - acc: 0.5452 - macro_f1score: 0.4632 - weighted_f1score: 0.0747

Accuracy: 0.5452, Macro F1 Score: 0.4632, Weighted F1 Score: 0.0747


### 4) My VGG11 (Convergence Speed Comparison)

In [0]:
# Vgg11에서, pretraining 된 fc층을 가지고 VGG 11을 재학습 시킬때, 얼마나 적은 epoch만에 수렴하는지를 보기 위해 만들었다.

def VGG11_init(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretrained)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
model = VGG11_init(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [75]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 12, 12, 32) 

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [77]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 6s 219us/sample - loss: 1.7563 - acc: 0.2834 - macro_f1score: 0.0216 - weighted_f1score: 0.0037 - val_loss: 1.6091 - val_acc: 0.3773 - val_macro_f1score: 0.1047 - val_weighted_f1score: 0.0206
Epoch 2/100
28698/28698 [==============================] - 6s 198us/sample - loss: 1.5347 - acc: 0.4050 - macro_f1score: 0.1446 - weighted_f1score: 0.0276 - val_loss: 1.4790 - val_acc: 0.4263 - val_macro_f1score: 0.1604 - val_weighted_f1score: 0.0307
Epoch 3/100
28698/28698 [==============================] - 6s 198us/sample - loss: 1.4414 - acc: 0.4431 - macro_f1score: 0.1880 - weighted_f1score: 0.0351 - val_loss: 1.3995 - val_acc: 0.4553 - val_macro_f1score: 0.1915 - val_weighted_f1score: 0.0349
Epoch 4/100
28698/28698 [==============================] - 6s 198us/sample - loss: 1.3616 - acc: 0.4744 - macro_f1score: 0.2269 - weighted_f1score: 0.0415 - val_loss: 1.3597 - val_acc: 0.4681 - val_

In [78]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 78us/sample - loss: 1.9521 - acc: 0.5170 - macro_f1score: 0.4541 - weighted_f1score: 0.0724

Accuracy: 0.5170, Macro F1 Score: 0.4541, Weighted F1 Score: 0.0724


## 4. For Size = 48, No Early Stopping
---

### 1) Epoch = 50

#### (1) My VGG11

In [0]:
############# 이 아래는 early stopping 없이 진행 #######################

In [0]:
# VGG 11 - pretraining
model = VGG11(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [82]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 6s 225us/sample - loss: 1.8184 - acc: 0.2473 - macro_f1score: 1.7316e-04 - weighted_f1score: 5.2760e-05 - val_loss: 1.8089 - val_acc: 0.2452 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
28698/28698 [==============================] - 6s 199us/sample - loss: 1.6620 - acc: 0.3311 - macro_f1score: 0.0767 - weighted_f1score: 0.0180 - val_loss: 1.5587 - val_acc: 0.3901 - val_macro_f1score: 0.1024 - val_weighted_f1score: 0.0216
Epoch 3/50
28698/28698 [==============================] - 6s 200us/sample - loss: 1.4892 - acc: 0.4164 - macro_f1score: 0.1607 - weighted_f1score: 0.0315 - val_loss: 1.4509 - val_acc: 0.4280 - val_macro_f1score: 0.1508 - val_weighted_f1score: 0.0296
Epoch 4/50
28698/28698 [==============================] - 6s 199us/sample - loss: 1.3959 - acc: 0.4527 - macro_f1score: 0.1969 - weighted_f1score: 0.0375 - val_loss: 1.3861 - val_acc: 0

In [83]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 83us/sample - loss: 3.3557 - acc: 0.5295 - macro_f1score: 0.4700 - weighted_f1score: 0.0742

Accuracy: 0.5295, Macro F1 Score: 0.4700, Weighted F1 Score: 0.0742


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

#### (2) My VGG16

In [0]:
# VGG 16 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG16(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [87]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 8s 263us/sample - loss: 1.8251 - acc: 0.2452 - macro_f1score: 2.6355e-04 - weighted_f1score: 7.0006e-05 - val_loss: 1.7996 - val_acc: 0.2488 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
28698/28698 [==============================] - 7s 231us/sample - loss: 1.7855 - acc: 0.2561 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8198 - val_acc: 0.2471 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
28698/28698 [==============================] - 7s 232us/sample - loss: 1.7232 - acc: 0.3051 - macro_f1score: 0.0187 - weighted_f1score: 0.0038 - val_loss: 1.6258 - val_acc: 0.3670 - val_macro_f1score: 0.0739 - val_weighted_f1score: 0.0169
Epoch 4/50
28698/28698 [==============================] - 7s 231us/sample - loss: 1.5709 - acc: 0.3898 - macro_f1score: 0.1201 - weighted_f1score: 0.0241 - val_loss: 1.5

In [88]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 92us/sample - loss: 2.8827 - acc: 0.5226 - macro_f1score: 0.4640 - weighted_f1score: 0.0729

Accuracy: 0.5226, Macro F1 Score: 0.4640, Weighted F1 Score: 0.0729


#### (3) My VGG19

In [0]:
# VGG 19 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [91]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 8s 277us/sample - loss: 1.8258 - acc: 0.2461 - macro_f1score: 1.6791e-04 - weighted_f1score: 3.1484e-05 - val_loss: 1.7979 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
28698/28698 [==============================] - 7s 238us/sample - loss: 1.7876 - acc: 0.2508 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.7777 - val_acc: 0.2616 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
28698/28698 [==============================] - 7s 240us/sample - loss: 1.7294 - acc: 0.2996 - macro_f1score: 0.0099 - weighted_f1score: 0.0013 - val_loss: 1.6838 - val_acc: 0.3249 - val_macro_f1score: 0.0613 - val_weighted_f1score: 0.0096
Epoch 4/50
28698/28698 [==============================] - 7s 243us/sample - loss: 1.6142 - acc: 0.3709 - macro_f1score: 0.1043 - weighted_f1score: 0.0200 - val_loss: 1.5

In [92]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 93us/sample - loss: 2.2941 - acc: 0.5256 - macro_f1score: 0.4585 - weighted_f1score: 0.0729

Accuracy: 0.5256, Macro F1 Score: 0.4585, Weighted F1 Score: 0.0729


### 2) Epoch = 100

#### (1) My VGG11

In [0]:
# VGG 11 - pretraining
model = VGG11(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [95]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 7s 237us/sample - loss: 1.8067 - acc: 0.2548 - macro_f1score: 0.0029 - weighted_f1score: 6.6780e-04 - val_loss: 1.7465 - val_acc: 0.2937 - val_macro_f1score: 0.0102 - val_weighted_f1score: 0.0025
Epoch 2/100
28698/28698 [==============================] - 6s 203us/sample - loss: 1.5981 - acc: 0.3640 - macro_f1score: 0.1171 - weighted_f1score: 0.0237 - val_loss: 1.5135 - val_acc: 0.4074 - val_macro_f1score: 0.1471 - val_weighted_f1score: 0.0277
Epoch 3/100
28698/28698 [==============================] - 6s 203us/sample - loss: 1.4631 - acc: 0.4248 - macro_f1score: 0.1686 - weighted_f1score: 0.0322 - val_loss: 1.4323 - val_acc: 0.4464 - val_macro_f1score: 0.1986 - val_weighted_f1score: 0.0376
Epoch 4/100
28698/28698 [==============================] - 6s 201us/sample - loss: 1.3813 - acc: 0.4600 - macro_f1score: 0.2048 - weighted_f1score: 0.0385 - val_loss: 1.3657 - val_acc: 0.4731 - 

In [96]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 80us/sample - loss: 3.7411 - acc: 0.5323 - macro_f1score: 0.4781 - weighted_f1score: 0.0740

Accuracy: 0.5323, Macro F1 Score: 0.4781, Weighted F1 Score: 0.0740


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

#### (2) My VGG16

In [0]:
# VGG 16 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG16(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [100]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 8s 273us/sample - loss: 1.8107 - acc: 0.2467 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.7765 - val_acc: 0.2644 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 7s 235us/sample - loss: 1.6865 - acc: 0.3130 - macro_f1score: 0.0565 - weighted_f1score: 0.0082 - val_loss: 1.6329 - val_acc: 0.3792 - val_macro_f1score: 0.1328 - val_weighted_f1score: 0.0266
Epoch 3/100
28698/28698 [==============================] - 7s 234us/sample - loss: 1.5222 - acc: 0.4002 - macro_f1score: 0.1564 - weighted_f1score: 0.0292 - val_loss: 1.4669 - val_acc: 0.4316 - val_macro_f1score: 0.1609 - val_weighted_f1score: 0.0300
Epoch 4/100
28698/28698 [==============================] - 7s 235us/sample - loss: 1.4193 - acc: 0.4417 - macro_f1score: 0.1890 - weighted_f1score: 0.0351 - val_loss: 1.3814 - val_ac

In [101]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 91us/sample - loss: 3.1536 - acc: 0.5368 - macro_f1score: 0.4930 - weighted_f1score: 0.0765

Accuracy: 0.5368, Macro F1 Score: 0.4930, Weighted F1 Score: 0.0765


#### (3) My VGG19

In [0]:
# VGG 19 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [42]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 7s 252us/sample - loss: 1.8134 - acc: 0.2458 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.7944 - val_acc: 0.2505 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 7s 231us/sample - loss: 1.7466 - acc: 0.2793 - macro_f1score: 0.0204 - weighted_f1score: 0.0024 - val_loss: 1.6955 - val_acc: 0.3073 - val_macro_f1score: 0.0524 - val_weighted_f1score: 0.0062
Epoch 3/100
28698/28698 [==============================] - 7s 232us/sample - loss: 1.5827 - acc: 0.3751 - macro_f1score: 0.1277 - weighted_f1score: 0.0240 - val_loss: 1.5140 - val_acc: 0.4035 - val_macro_f1score: 0.1532 - val_weighted_f1score: 0.0294
Epoch 4/100
28698/28698 [==============================] - 7s 232us/sample - loss: 1.4827 - acc: 0.4187 - macro_f1score: 0.1622 - weighted_f1score: 0.0305 - val_loss: 1.4554 - val_ac

In [43]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 90us/sample - loss: 3.1279 - acc: 0.5348 - macro_f1score: 0.4754 - weighted_f1score: 0.0751

Accuracy: 0.5348, Macro F1 Score: 0.4754, Weighted F1 Score: 0.0751


## 5. Model Save & Import
---

In [0]:
#참고.
model.save('CNN model/VGG19.h5') # 모델 저장

In [0]:
model.save_weights('CNN model/VGG19_weights.h5') # 가중치 저장

In [0]:
model2 = load_model('CNN model/VGG19.h5',custom_objects={"macro_f1score": macro_f1score,"weighted_f1score":weighted_f1score}) # 모델 불러오기  custom_objects={"":} 로 정의했던 측도들 불러온다.